In [1]:
!brew install ffmpeg

zsh:1: command not found: brew


In [2]:
import os

# Add the directory containing ffmpeg to the PATH in the notebook
os.environ['PATH'] += os.pathsep + '/usr/local/bin'

# Check if ffmpeg is now recognized
!which ffmpeg


/usr/local/bin/ffmpeg


In [3]:
!which ffprobe

/usr/local/bin/ffprobe


In [4]:
import os
from os import environ
import yt_dlp
import openai
from pydub import AudioSegment
from pyannote.audio import Pipeline
from openai import OpenAI

client = OpenAI()

# Set your OpenAI API key
client.api_key = os.environ["OPENAI_API_KEY"]

# Hugging Face API key
HF_API_Key = os.environ["HF_API_KEY"]

```python
 # Specify the location of ffmpeg
        'ffmpeg_location': '/usr/local/bin/ffmpeg',  # Use the path found via `which ffmpeg`
        'outtmpl': output_file
```        

In [ ]:
import yt_dlp
import subprocess
import os

urls = [
        "https://www.youtube.com/",
        "https://www.youtube.com/",
        "https://www.youtube.com/"
    ]

def download_and_convert(url):
    try:
        ydl_opts = {
            'format': 'bestaudio/best',
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'wav',
            }],
            'outtmpl': '%(title)s.%(ext)s',
        }

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=True)
            filename = ydl.prepare_filename(info).replace('.webm', '.wav')

        # Convert to 16kHz mono
        output_filename = f"{os.path.splitext(filename)[0]}_16khz_mono.wav"
        cmd = [
            "ffmpeg",
            "-i", filename,
            "-ar", "16000",
            "-ac", "1",
            "-c:a", "pcm_s16le",
            output_filename
        ]
        subprocess.run(cmd, check=True)

        # Clean up original WAV file
        os.remove(filename)

        print(f"Processed: {output_filename}")
    except Exception as e:
        print(f"Error processing {url}: {str(e)}")

for url in urls:
    download_and_convert(url)

print("All videos processed!")

In [ ]:
!pip install openai pyannote.audio


In [ ]:
import os
from os import environ
import openai
from openai import OpenAI
from pyannote.audio import Pipeline

client = OpenAI()

# Set your OpenAI API key
client.api_key = os.environ["OPENAI_API_KEY"]

# Set your Hugging Face API key
HF_API_Key = os.environ["HF_API_KEY"]

# Initialize the speaker diarization pipeline
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization")

# Path to the directory containing the .wav files
wav_directory = "~/wd/GitHub/Research/Ops-Research/audio_files"  # Adjust this path as needed

def transcribe_audio_with_speaker_diarization(audio_file_path):
    # Perform transcription using OpenAI's Whisper
    with open(audio_file_path, 'rb') as audio_file:
        transcript = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file)
    print(transcript.text)
    
    # Perform speaker diarization using pyannote.audio
    diarization = pipeline({'uri': audio_file_path, 'audio': audio_file_path})

    # Combine transcription with speaker labels
    transcribed_text = transcript['text']
    speaker_segments = []

    for segment, track, label in diarization.itertracks(yield_label=True):
        speaker_segments.append((segment.start, segment.end, label))
    
    labeled_transcription = []

    for start, end, speaker in speaker_segments:
        # Match the speaker segments with the transcription (approximate)
        labeled_transcription.append(f"Speaker {speaker}: {transcribed_text}")
    
    return "\n".join(labeled_transcription)

def process_wav_files(directory):
    for file_name in os.listdir(directory):
        if file_name.endswith("_16khz_mono.wav"):
            audio_file_path = os.path.join(directory, file_name)
            transcription_file = os.path.splitext(audio_file_path)[0] + "_transcription.txt"
            
            transcription_text = transcribe_audio_with_speaker_diarization(audio_file_path)
            
            with open(transcription_file, 'w') as f:
                f.write(transcription_text)
            
            print(f"Transcription saved: {transcription_file}")

if __name__ == "__main__":
    process_wav_files(wav_directory)


In [ ]:
import os
import openai
from pyannote.audio import Pipeline
from openai import OpenAI
from os import environ

# Set your OpenAI API key
client.api_key = os.environ["OPENAI_API_KEY"]

# Set your Hugging Face API key (if needed for pyannote)
HF_API_Key = os.environ["HF_API_KEY"]

# Initialize the speaker diarization pipeline
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization@2.1", use_auth_token=HF_API_Key)

# Path to the directory containing the .wav files
wav_directory = "~/wd/GitHub/Research/Ops-Research/"  # Adjust this path as needed

audio_file_path = "~/wd/GitHub/Research/Ops-Research/audio_files/liron_audio_1.wav"
#transcription_file = "~/wd/GitHub/Research/Ops-Research/transcription_files/liron_audio_1_transcription.txt"

def transcribe_audio_with_speaker_diarization(audio_file_path):
    # Perform transcription using OpenAI's Whisper
    with open(audio_file_path, 'rb') as audio_file:
        transcript = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file)
    print(transcript.text)
    # Perform speaker diarization using pyannote.audio
    diarization = pipeline({'uri': audio_file_path, 'audio': audio_file_path})

    # Combine transcription with speaker labels
    transcribed_text = transcript['text']
    speaker_segments = []

    for segment, track, label in diarization.itertracks(yield_label=True):
        speaker_segments.append((segment.start, segment.end, label))
    
    labeled_transcription = []

    for start, end, speaker in speaker_segments:
        # Match the speaker segments with the transcription (approximate)
        labeled_transcription.append(f"Speaker {speaker}: {transcribed_text}")
    
    return "\n".join(labeled_transcription)

def process_wav_files(directory):
    for file_name in os.listdir(directory):
        if file_name.endswith(".wav"):
            audio_file_path = os.path.join(directory, file_name)
            transcription_file = os.path.splitext(audio_file_path)[0] + "_transcription.txt"
            
            transcription_text = transcribe_audio_with_speaker_diarization(audio_file_path)
            
            with open(transcription_file, 'w') as f:
                f.write(transcription_text)
            
            print(f"Transcription saved: {transcription_file}")

if __name__ == "__main__":
    process_wav_files(wav_directory)


In [ ]:
# Transcribe the .wav file to a .srt file with speaker labels
import os
import openai
from pyannote.audio import Pipeline
from openai import OpenAI
from os import environ

# Set your OpenAI API key
client.api_key = os.environ["OPENAI_API_KEY"]

# Set your Hugging Face API key (if needed for pyannote)
HF_API_Key = os.environ["HF_API_KEY"]

# Initialize the speaker diarization pipeline
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization@2.1", use_auth_token=HF_API_Key)

# Path to the directory containing the .wav files
wav_directory = "~/wd/GitHub/Research/Ops-Research/"  # Adjust this path as needed

audio_file_path = "~/wd/GitHub/Research/Ops-Research/audio_files/audio.wav"
#transcription_file = "~/wd/GitHub/Research/Ops-Research/transcription_files

def transcribe_audio_with_speaker_diarization(audio_file_path):
    # Perform transcription using OpenAI's Whisper
    with open(audio_file_path, 'rb') as audio_file:
        transcript = client.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file)
    print(transcript.txt)
    # Perform speaker diarization using pyannote.audio
    diarization = pipeline({'uri': audio_file_path, 'audio': audio_file_path})

    # Combine transcription with speaker labels
    transcribed_text = transcript['text']   
    speaker_segments = []

    for segment, track, label in diarization.itertracks(yield_label=True):
        speaker_segments.append((segment.start, segment.end, label))   
    labeled_transcription = []

    for start, end, speaker in speaker_segments:
        # Match the speaker segments with the transcription (approximate)
        labeled_transcription.append(f"Speaker {speaker}: {transcribed_text}")

    return "\n".join(labeled_transcription)

def process_wav_files(directory):
    for file_name in os.listdir(directory):
        if file_name.endswith(".wav"):
            audio_file_path = os.path.join(directory, file_name)
            transcription_file = os.path.splitext(file_name + "_transcription.txt") 
    print(transcription_file)

# Create a .srt file with the transcribed text
def create_srt_file(transcription_file):
    with open(transcription_file, 'r') as f:
        lines = f.readlines()
    
    srt_file = os.path.splitext(transcription_file)[0] + ".srt"
    with open(srt_file, 'w') as f:
        for i, line in enumerate(lines):
            f.write(f"{i+1}\n")
            f.write(line)
            f.write("\n")

return srt_file

print(f"SRT file created: {srt_file}")

srt_file_path = "~/wd/GitHub/Research/Ops-Research/srt_files/audio.srt"

srt_file.save(srt_file_path)

print(f"SRT file saved: {srt_file_path}")

if __name__ == "__main__":
    process_wav_files(wav_directory)
    create_srt_file(transcription_file)    

# This is a complete mess and absolute worthless code that is mainly storing thoughts for the code that will be written in the future.
